In [1]:
import pandas as pd
import numpy as np
import ast
import json
import re

# cargo el archivo para su lectura
eda = pd.read_csv(r"C:\Users\migue\OneDrive\Escritorio\movies_dataset.csv")

# Eliminación primeras columnas que no serán utilizadas
columnas_eliminar = ['video', "imdb_id", "adult", "original_title", "poster_path", "homepage"]
eda.drop(columns=columnas_eliminar, inplace=True)

# Se eliminan 87 entradas que no tienen release_date
eda.dropna(subset=['release_date'], inplace=True)

#Se deciden eliminar las 3 entradas de revenue 0 por data insuficiente
eda.dropna(subset=['revenue'], inplace=True)


# Creamos columa return (revenue/budget) que dará 0 en caso de no poder realizar la división
#primero creamos la función que calculará el return
def cal_return(revenue, budget):
    if float(budget) == 0:
        return 0
    else:
        return float(revenue)/float(budget)
    
#creamos nueva columa return
eda['return']=eda.apply(lambda row: cal_return(row['revenue'], row['budget']), axis=1)


# transformamos las fechas de release_date a datetime
eda['release_date']=pd.to_datetime(eda['release_date'])

# eda['release_date']=eda['release_date'].dt.strftime('%Y-%m-%d')  #Esto es para volverlo str con formato AAAA-mm-dd

# se crea columna 'release_year' donde se alberga solo el año de estreno
eda['release_year']=eda['release_date'].dt.year



                # DESANIDADO


# Se crea una función que reemplaza las comillas ' por " para manejar json
def reemplazar_comillas(cadena):
  return re.sub(r"'", '"', cadena)

#Se cambian comillas de columna genres
eda['genres'] = eda['genres'].apply(reemplazar_comillas)

#Se transforma los datos de str a listas
eda['genres'] = eda['genres'].apply(lambda x: json.loads(x))

#Se desanidan id y los nombres de la columna genres
eda['genres_ids'] = eda['genres'].apply(lambda x: [d['id'] for d in x])
eda['genres_names'] = eda['genres'].apply(lambda x: [d['name'] for d in x])


# funcion para formatear las conlumnas con listas de diccionarios
def convert_to_list_of_dicts(value):
    try:
        value = value.strip()
        return ast.literal_eval(value)
    except (ValueError, SyntaxError):
        return [] 

# Se transforma la columna production_companies y se desanida en id y names
eda['production_companies'] = eda['production_companies'].apply(convert_to_list_of_dicts)
eda['production_companies_id'] = eda['production_companies'].apply(lambda x: [d['id'] for d in x])
eda['production_companies_names'] = eda['production_companies'].apply(lambda x: [d['name'] for d in x])


# de la columna production_countries se desanida el codigo iso
eda['production_countries'] = eda['production_countries'].apply(convert_to_list_of_dicts)
eda['production_countries_iso'] = eda['production_countries'].apply(lambda x: [d['iso_3166_1'] for d in x])
eda['production_countries_names'] = eda['production_countries'].apply(lambda x: [d['name'] for d in x])

# se transforma la columna spoken_languages a lista de dicc y se desanidan
eda['spoken_languages'] = eda['spoken_languages'].apply(convert_to_list_of_dicts)
eda['spoken_languages_iso'] = eda['spoken_languages'].apply(lambda x: [d['iso_639_1'] for d in x])
eda['spoken_languages_name'] = eda['spoken_languages'].apply(lambda x: [d['name'] for d in x])



# Se reemplazan los Nan de belongs_to_collection por dicc vacios
#eda['belongs_to_collection']=eda['belongs_to_collection'].fillna('{}') #opcion 1 funciona pero son str y falla luego
eda['belongs_to_collection']=eda['belongs_to_collection'].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else {})

# se crean funciones para extraer id y name de los diccionarios
def extract_id(row):
    try:
        return row['id']
    except:
        return None

def extract_name(row):
    try:
        return row['name']
    except:
        return None
    
# se crean nuevas columnas con los id y name extraidos
eda['belongs_to_collection_id']=eda['belongs_to_collection'].apply(extract_id)
eda['belongs_to_collection_name']=eda['belongs_to_collection'].apply(extract_name)


# Eliminación columnas desanidadas
columnas_eliminar = ['belongs_to_collection', "genres", "production_countries", "production_companies", "spoken_languages"]
eda.drop(columns=columnas_eliminar, inplace=True)


C:\Users\migue\AppData\Local\Temp\ipykernel_11428\79583190.py:8: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  eda = pd.read_csv(r"C:\Users\migue\OneDrive\Escritorio\movies_dataset.csv")


In [2]:
eda.head(3)

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,...,genres_ids,genres_names,production_companies_id,production_companies_names,production_countries_iso,production_countries_names,spoken_languages_iso,spoken_languages_name,belongs_to_collection_id,belongs_to_collection_name
0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,...,"[16, 35, 10751]","[Animation, Comedy, Family]",[3],[Pixar Animation Studios],[US],[United States of America],[en],[English],10194.0,Toy Story Collection
1,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,...,"[12, 14, 10751]","[Adventure, Fantasy, Family]","[559, 2550, 10201]","[TriStar Pictures, Teitler Film, Interscope Co...",[US],[United States of America],"[en, fr]","[English, Français]",NaN,None
2,0,15602,en,A family wedding reignites the ancient feud be...,11.7129,1995-12-22,0.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,...,"[10749, 35]","[Romance, Comedy]","[6194, 19464]","[Warner Bros., Lancaster Gate]",[US],[United States of America],[en],[English],119050.0,Grumpy Old Men Collection


In [3]:


# Se carga el segundo archivo
eda2 = pd.read_csv(r"C:\Users\migue\OneDrive\Escritorio\credits.csv")

# se formatea columna cast y se desanida en los actores/actrices
eda2['cast'] = eda2['cast'].apply(convert_to_list_of_dicts)
eda2['cast_names'] = eda2['cast'].apply(lambda x: [d['name'] for d in x])

# se crea una función para extraer director de la columna crew
def extraer_nombre_director(lista_diccionarios):
  for diccionario in lista_diccionarios:
    if diccionario['job'] == 'Director':
      return diccionario['name']
  return None

# Se transforma crew a lista de dicc y se le aplica el extractor de director para crear nueva columna
eda2['crew'] = eda2['crew'].apply(convert_to_list_of_dicts)
eda2['director_name']=eda2['crew'].apply(extraer_nombre_director)

# se eliminan columnas cast y crew originales
columnas_eliminar = ['crew', 'cast']
eda2.drop(columns=columnas_eliminar, inplace=True)

# transformamos el dtype de columna 'id' a int64 para poder hacer merge
eda['id'] = eda['id'].astype('int64')

# Merge de eda y eda2 en un solo df
df = eda.merge(eda2, on='id', how='inner')

In [4]:
df.head(3)

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,...,production_companies_id,production_companies_names,production_countries_iso,production_countries_names,spoken_languages_iso,spoken_languages_name,belongs_to_collection_id,belongs_to_collection_name,cast_names,director_name
0,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,...,[3],[Pixar Animation Studios],[US],[United States of America],[en],[English],10194.0,Toy Story Collection,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...",John Lasseter
1,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,...,"[559, 2550, 10201]","[TriStar Pictures, Teitler Film, Interscope Co...",[US],[United States of America],"[en, fr]","[English, Français]",NaN,None,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...",Joe Johnston
2,0,15602,en,A family wedding reignites the ancient feud be...,11.7129,1995-12-22,0.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,...,"[6194, 19464]","[Warner Bros., Lancaster Gate]",[US],[United States of America],[en],[English],119050.0,Grumpy Old Men Collection,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...",Howard Deutch


In [5]:
print(df.isna().sum())

budget                            0
id                                0
original_language                11
overview                        941
popularity                        0
release_date                      0
revenue                           0
runtime                         246
status                           80
tagline                       25026
title                             0
vote_average                      0
vote_count                        0
return                            0
release_year                      0
genres_ids                        0
genres_names                      0
production_companies_id           0
production_companies_names        0
production_countries_iso          0
production_countries_names        0
spoken_languages_iso              0
spoken_languages_name             0
belongs_to_collection_id      40954
belongs_to_collection_name    40954
cast_names                        0
director_name                   835
dtype: int64


In [6]:
df.shape

(45451, 27)

In [7]:
df.to_csv('limpio1.csv', index=False)